In [95]:
from dotenv import load_dotenv
load_dotenv()

True

In [96]:
from langchain_community.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import requests

In [97]:
## tool create

from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches the currency conversion factor between a given base currency and a target currency"""

    url = f"https://v6.exchangerate-api.com/v6/97fcb21b253f902d8ed799f5/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """
    Given a currency conversion rate, this function calculates the target currency value from a given base currency value
    """

    return base_currency_value*conversion_rate

In [98]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1753401601,
 'time_last_update_utc': 'Fri, 25 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1753488001,
 'time_next_update_utc': 'Sat, 26 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 86.4296}

In [99]:
convert.invoke({'base_currency_value':10,'conversion_rate':86.42})

864.2

In [100]:
## tool binding

llm = ChatOpenAI()

In [101]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [118]:
messages = [HumanMessage('can you convert 72 indian rupess to usd and also tell What is the conversion factor between them')]

In [119]:
messages

[HumanMessage(content='can you convert 72 indian rupess to usd and also tell What is the conversion factor between them', additional_kwargs={}, response_metadata={})]

In [120]:
ai_message = llm_with_tools.invoke(messages)

In [121]:
messages.append(ai_message)

In [122]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_OOPw2Wy1oQ97mfUsb6cC44hw',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 72},
  'id': 'call_1U8uY5wdGh8NQibfq4ebRAwS',
  'type': 'tool_call'}]

In [123]:
import json
for tool_call in ai_message.tool_calls:
    # execute the first tool and get the value of conversion rate

    if tool_call['name'] == 'get_conversion_factor':
        tool_msg1 = get_conversion_factor.invoke(tool_call)

        # fetch this conversion rate
        conversion_rate = json.loads(tool_msg1.content)['conversion_rate']

        # append this tool message to message list
        messages.append(tool_msg1)

    # execute the second tool using the conversion rate from tool 1

    if tool_call['name'] == 'convert':
        ## fetch the current arg
        tool_call['args']['conversion_rate'] = conversion_rate

        tool_msg2 = convert.invoke(tool_call)

        messages.append(tool_msg2)

In [124]:
messages

[HumanMessage(content='can you convert 72 indian rupess to usd and also tell What is the conversion factor between them', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OOPw2Wy1oQ97mfUsb6cC44hw', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_1U8uY5wdGh8NQibfq4ebRAwS', 'function': {'arguments': '{"base_currency_value": 72}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 121, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BxAa7FvL82ZU47cdehMMJ4fYDHhId', 'service_tier': 'defaul

In [125]:
llm_with_tools.invoke(messages).content

'The conversion factor between Indian Rupee (INR) and US Dollar (USD) is 0.01157. \n\nWhen you convert 72 Indian Rupees (INR) to US Dollars (USD), it equals approximately $0.83.'